In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15509654
paper_name = 'perrone_dawes_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/supp_tables.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 277 x 3


In [7]:
original_data['orf'] = original_data['orf'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [9]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                orf gene  fold
index_input                   
89           SNARES  NaN   NaN


In [11]:
original_data = original_data.loc[t,:]

In [12]:
original_data = original_data.loc[original_data['fold'].notnull(),:]

In [13]:
# Transforming into log-scale, so that WT (and all other mutants) have a value of 0 (fold = 1).
original_data['data'] = pd.to_numeric(original_data['fold'], errors='coerce')
original_data['data'] = np.log2(original_data['data'].values)

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(268, 1)

# Prepare the final dataset

In [18]:
data = original_data[['data']].copy()

In [19]:
dataset_ids = [16434]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,16434
data_type,value
orf,
YAL002W,2.807355
YAL024C,1.000000
YAL047C,2.321928
YBL007C,3.169925
YBL009W,2.584963


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16434
,data_type,value
gene_id,orf,
2,YAL002W,2.807355
22,YAL024C,1.000000
45,YAL047C,2.321928
95,YBL007C,3.169925
97,YBL009W,2.584963


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16434          
data_type           value    valuez
gene_id orf                        
2       YAL002W  2.807355  4.053642
22      YAL024C  1.000000  1.443936
45      YAL047C  2.321928  3.352716
95      YBL007C  3.169925  4.577170
97      YBL009W  2.584963  3.732521

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15509654...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

Updating the data_modified_on field...
